# Production Runbook


In [ ]:
import sys, subprocess
from pathlib import Path

print("Python:", sys.executable)
repo_root = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
subprocess.run(["pip", "install", "-e", str(repo_root)], check=True)


In [ ]:
from pathlib import Path
import shutil

raw_csv = repo_root / "data" / "raw" / "time_series_60min_singleindex.csv"
if not raw_csv.exists():
    nested = next((repo_root / "data" / "raw").glob("opsd-time_series-*/time_series_60min_singleindex.csv"), None)
    if nested:
        shutil.copy(nested, raw_csv)
        print("Copied:", nested, "->", raw_csv)
    else:
        print("Missing OPSD CSV. Please download or place it in data/raw/")
print("Raw CSV:", raw_csv)


In [ ]:
import subprocess

subprocess.run([
    "python", "-m", "gridpulse.data_pipeline.validate_schema",
    "--in", "data/raw", "--report", "reports/data_quality_report.md"
], check=True, cwd=str(repo_root))

subprocess.run([
    "python", "-m", "gridpulse.data_pipeline.build_features",
    "--in", "data/raw", "--out", "data/processed"
], check=True, cwd=str(repo_root))

subprocess.run([
    "python", "-m", "gridpulse.data_pipeline.split_time_series",
    "--in", "data/processed/features.parquet",
    "--out", "data/processed/splits"
], check=True, cwd=str(repo_root))


End‑to‑end runbook for production‑style execution.

## 1) Data → Features → Splits

In [4]:
from pathlib import Path
import shutil

raw_csv = repo_root / "data" / "raw" / "time_series_60min_singleindex.csv"
if not raw_csv.exists():
    nested = next((repo_root / "data" / "raw").glob("opsd-time_series-*/time_series_60min_singleindex.csv"), None)
    if nested:
        shutil.copy(nested, raw_csv)
        print("Copied:", nested, "->", raw_csv)
    else:
        print("Missing OPSD CSV. Please download or place it in data/raw/")
print("Raw CSV:", raw_csv)


Raw CSV: /Users/pratik_n/Downloads/gridpulse/data/raw/time_series_60min_singleindex.csv


## 2) Baselines + GBM

## 3) Deep models (LSTM + TCN)

In [6]:
import subprocess

subprocess.run(['python', '-m', 'gridpulse.forecasting.train', '--config', 'configs/train_forecast.yaml'], check=True)


Traceback (most recent call last):
  File "/Applications/Xcode.app/Contents/Developer/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Applications/Xcode.app/Contents/Developer/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/Users/pratik_n/Downloads/gridpulse/src/gridpulse/forecasting/train.py", line 322, in <module>
    main()
  File "/Users/pratik_n/Downloads/gridpulse/src/gridpulse/forecasting/train.py", line 160, in main
    cfg = yaml.safe_load(Path(args.config).read_text(encoding="utf-8"))
  File "/Applications/Xcode.app/Contents/Developer/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/pathlib.py", line 1256, in read_text
    with self.open(mode='r', encoding=encoding, errors=errors) as f:
  File "/Applications/Xcode.app/Contents/Developer/Library/Frameworks/Python3

CalledProcessError: Command '['python', '-m', 'gridpulse.forecasting.train', '--config', 'configs/train_forecast.yaml']' returned non-zero exit status 1.

## 4) Start API + Dashboard (run in terminals)

- `uvicorn services.api.main:app --reload --port 8000`
- `streamlit run services/dashboard/app.py`


## 5) Monitoring + Optimization

- `curl http://localhost:8000/monitor`
- `curl -X POST http://localhost:8000/optimize -H 'Content-Type: application/json' -d '{"forecast_load_mw":[8000,8200],"forecast_renewables_mw":[3200,3100]}'`
